# Creating pulses

This notebook will illustrate how to create different kinds of pulses for the beamline.

In [ ]:
import numpy as np
import scipp as sc
import tof

AA = sc.Unit('angstrom')
ms = sc.Unit('ms')

## A basic pulse

Pulses are characterized by three aspects:

- the number of neutrons in the pulse
- the time at which each neutron is born inside the pulse
- the wavelength of each neutron inside the pulse

By default, the time and wavelength distributions of neutrons are both flat (uniform).
We then need to define a pulse duration (start and end times) and a wavelength range.

To create a pulse with 1M neutrons, uniformly distributed in the ranges of 1-3 ms for birth times,
and 1-10 Å for wavelengths, we write:

In [ ]:
pulse = tof.Pulse(
    neutrons=1_000_000, tmin=1.0 * ms, tmax=3.0 * ms, lmin=1.0 * AA, lmax=10.0 * AA
)
pulse

To inspect the data in the pulse, we can either look at it `.birth_times` or `.wavelengths` members

In [ ]:
pulse.wavelengths

or we can plot it with

In [ ]:
pulse.plot()

## Specifying time and wavelength distributions

Pulses at neutron facilities are never flat.
To change the time and wavelength distributions of the neutrons in your pulse,
you can specify arrays of probabilities.
The array values represent the probabilities, while the associated coordinate represent the values to be sampled from.

As an example, we create a triangular distribution for the neutron birth times,
and a distribution with only 4 possible values for the neutron wavelengths.

In [ ]:
v = np.arange(30.0)
p_time = sc.DataArray(
    data=sc.array(dims=['time'], values=np.concatenate([v, v[::-1]])),
    coords={'time': sc.linspace('time', 0.1, 6.0, len(v) * 2, unit='ms')},
)
p_wav = sc.DataArray(
    data=sc.array(dims=['wavelength'], values=[1.0, 2.0, 3.0, 4.0]),
    coords={
        'wavelength': sc.array(
            dims=['wavelength'], values=[1.0, 2.0, 3.0, 4.0], unit='angstrom'
        )
    },
)

pulse = tof.Pulse.from_distribution(neutrons=200_000, p_time=p_time, p_wav=p_wav)
pulse.plot()

It is possible to use a finer sampling of the supplied distribution by setting the `sampling` parameter:

In [ ]:
pulse = tof.Pulse.from_distribution(
    neutrons=200_000, p_time=p_time, p_wav=p_wav, sampling=10000
)
pulse.plot()

## Using a pre-defined facility pulse

Time and wavelength distributions for pulses at neutron facilities can be much more complex than simple mathematical functions.
Some pre-defined pulse profiles are available via the `from_facility` method:

In [ ]:
pulse = tof.Pulse.from_facility('ess', neutrons=1_000_000)
pulse.plot()

In this case, it is possible to limit the time and wavelength ranges if needed:

In [ ]:
pulse = tof.Pulse.from_facility(
    'ess', neutrons=1_000_000, tmax=3.2 * ms, lmax=10.0 * AA
)
pulse.plot()

## Specifying neutrons manually

Finally, it is possible to simply specify a list of birth times and wavelengths manually to create a pulse via the `from_neutrons` method.

In [ ]:
pulse = tof.Pulse.from_neutrons(
    birth_times=sc.array(
        dims=['event'],
        values=[0.0, 0.1, 0.2, 0.56],
        unit='s',
    ),
    wavelengths=sc.array(dims=['event'], values=[5.0, 8.0, 11.0, 7.1], unit='angstrom'),
)
pulse.plot()